In [4]:
import os
import pandas as pd
import glob
import pickle
from selenium import webdriver
from selenium.webdriver.common.by import By
import concurrent.futures 
from pycaret.classification import *
from sklearn.metrics import accuracy_score
import requests
from bs4 import BeautifulSoup
import time
import random
from fuzzywuzzy import fuzz, process
from os import walk

In [54]:
filenames = next(walk('C:/Users\idotz/OneDrive - Northeastern University/Python/Soccer Analysis'), (None, None, []))[2]
filenames = [f for f in filenames if '2022' in f]

all_data = []
for f in filenames:
    open_file = open(f, "rb")
    d = pickle.load(open_file)
    all_data.append(d)
    open_file.close()
    
all_data = [i for j in all_data for i in j]

f = []
for dic in all_data:
    d = {}
    for k, v in dic.items():
        try:
            d[k] = float(v[0]) - float(v[1])
        except:
            d[k] = v
    f.append(d)

df = pd.DataFrame(f)
df = df[df['half_time'] == 'Half-Time']
df = df[(df['odds'] != '') & (df['odds'] != {})]
df = df.reset_index(drop = True)
df2 = pd.json_normalize(df['bet'])
df = pd.concat([df, df2], axis = 1)
df['o_names'] = df['odds_url'].apply(lambda x: ['draw'] + x.split('/')[-2].split('-v-'))
r = []
for i in df[['odds','o_names']].to_numpy():
    oo = [getname(name, list(i[0].keys())) for name in i[1]]
    oo = [i[0][key] for key in oo]
    r.append(oo)
df['oo'] = r
split_df = pd.DataFrame(df['oo'].tolist(), columns=['0', '1', '2'])
df = pd.concat([df, split_df], axis=1)
df = df[['url','draw','home','away','odds_checker_sum','winner','0','1','2']]
df = df.replace('no value', 0)

In [55]:
df = df[['url','draw','home','away','odds_checker_sum','winner','0','1','2']]

In [56]:
df = df.replace('no value', 0)

In [57]:
dha = {'draw':'0','home':'1','away':'2'}

In [58]:
df

,url,draw,home,away,odds_checker_sum,winner,0,1,2
0,https://www.fotmob.com/match/3888550/matchfact...,0.00000,0.07935,0.00000,1.091726,1,2.300000,2.387500,4.200000
1,https://www.fotmob.com/match/3888551/matchfact...,0.00000,0.00000,0.33750,1.087449,2,4.750000,13.000000,1.250000
2,https://www.fotmob.com/match/3900373/matchfact...,0.02265,0.00000,0.00000,1.044048,2,20.000000,56.000000,1.024390
3,https://www.fotmob.com/match/3904422/matchfact...,0.00000,0.02020,0.00000,1.064560,2,3.500000,6.500000,1.600000
4,https://www.fotmob.com/match/3902828/matchfact...,0.00000,0.00000,0.47400,0.970938,2,226.000000,201.000000,1.040000
5,https://www.fotmob.com/match/3916965/matchfact...,0.00000,0.00000,0.20420,1.092484,2,3.126923,4.500000,1.816667
6,https://www.fotmob.com/match/3887414/matchfact...,0.00000,0.00000,0.03310,1.086649,1,3.900000,1.444444,7.250000
7,https://www.fotmob.com/match/3887416/matchfact...,0.00000,0.00000,0.22605,1.079192,2,2.500000,2.050000,5.225000
8,https://www.fotmob.com/match/3905446/matchfact...,0.00000,0.00000,0.26260,1.085165,2,4.000000,7.000000,1.444444
9,https://www.fotmob.com/match/3900374/matchfact...,0.00910,0.00000,0.00000,1.009835,2,100.000000,251.000000,1.004167


In [59]:
df['ft_winner'] = scores

In [73]:
for word, number in dha.items():
    df[f'{word}_profits'] = np.where(df['ft_winner'] == int(number),
                                    df[word] * df[number] * 100 - df[word] * 100,
                                    -100*df[word])

In [80]:
df.sort_values(by = '1')

,url,draw,home,away,odds_checker_sum,winner,0,1,2,ft_winner,draw_profits,home_profits,away_profits
75,https://www.fotmob.com/match/3900981/matchfact...,0.00400,0.00000,0.00000,1.005880,1,251.000000,1.000100,501.000000,1,-0.40000,0.000000,-0.000000
193,https://www.fotmob.com/match/3917964/matchfact...,0.00155,0.00000,0.00000,1.015718,1,67.000000,1.003333,243.000000,1,-0.15500,0.000000,-0.000000
36,https://www.fotmob.com/match/3917948/matchfact...,0.02290,0.00000,0.00000,1.015256,1,67.000000,1.003667,251.000000,1,-2.29000,0.000000,-0.000000
136,https://www.fotmob.com/match/3900378/matchfact...,0.01295,0.00000,0.00000,1.021751,1,51.000000,1.004500,151.000000,1,-1.29500,0.000000,-0.000000
67,https://www.fotmob.com/match/3900361/matchfact...,0.00000,0.00000,0.00000,1.019505,1,56.000000,1.005000,151.000000,1,-0.00000,0.000000,-0.000000
58,https://www.fotmob.com/match/3915358/matchfact...,0.00000,0.00000,0.00000,1.027352,1,41.000000,1.005000,126.000000,1,-0.00000,0.000000,-0.000000
53,https://www.fotmob.com/match/3919112/matchfact...,0.01780,0.00000,0.00000,1.027447,1,34.000000,1.010000,126.000000,1,-1.78000,0.000000,-0.000000
91,https://www.fotmob.com/match/3900367/matchfact...,0.01360,0.00000,0.00000,1.028571,1,35.000000,1.010000,101.000000,1,-1.36000,0.000000,-0.000000
110,https://www.fotmob.com/match/3902849/matchfact...,0.00065,0.00000,0.00000,1.026886,1,36.000000,1.010000,111.000000,1,-0.06500,0.000000,-0.000000
189,https://www.fotmob.com/match/3917963/matchfact...,0.00705,0.00000,0.00000,1.034525,1,26.000000,1.015152,91.000000,1,-0.70500,0.000000,-0.000000


In [53]:
df['profit'] = np.where(df['othercolumn'] == 'CertainString',
                                           df['columnname'] * 1000,
                                           df['columnname'])

,url,draw,home,away,odds_checker_sum,winner,0,1,2,ft_winner
0,https://www.fotmob.com/match/3888550/matchfact...,0.00000,0.07935,0.00000,1.091726,1,2.300000,2.387500,4.200000,0
1,https://www.fotmob.com/match/3888551/matchfact...,0.00000,0.00000,0.33750,1.087449,2,4.750000,13.000000,1.250000,2
2,https://www.fotmob.com/match/3900373/matchfact...,0.02265,0.00000,0.00000,1.044048,2,20.000000,56.000000,1.024390,2
3,https://www.fotmob.com/match/3904422/matchfact...,0.00000,0.02020,0.00000,1.064560,2,3.500000,6.500000,1.600000,1
4,https://www.fotmob.com/match/3902828/matchfact...,0.00000,0.00000,0.47400,0.970938,2,226.000000,201.000000,1.040000,2
5,https://www.fotmob.com/match/3916965/matchfact...,0.00000,0.00000,0.20420,1.092484,2,3.126923,4.500000,1.816667,2
6,https://www.fotmob.com/match/3887414/matchfact...,0.00000,0.00000,0.03310,1.086649,1,3.900000,1.444444,7.250000,1
7,https://www.fotmob.com/match/3887416/matchfact...,0.00000,0.00000,0.22605,1.079192,2,2.500000,2.050000,5.225000,1
8,https://www.fotmob.com/match/3905446/matchfact...,0.00000,0.00000,0.26260,1.085165,2,4.000000,7.000000,1.444444,2
9,https://www.fotmob.com/match/3900374/matchfact...,0.00910,0.00000,0.00000,1.009835,2,100.000000,251.000000,1.004167,2


In [6]:
def getname(name, l):
    if len(l) == 0:
        return ''
    highest = process.extractOne(name,l)
    return highest[0]

In [7]:
def get_winner(url):
    page = requests.get(url)
    time.sleep(random.randint(0, 4))
    soup = BeautifulSoup(page.content, 'html.parser')
    score = soup.find_all("span", {"class": "css-bw7eig-topRow"})[0].text.split(' - ')
    score = int(score[0]) - int(score[1])
    score = winner(score)
    return score

In [8]:
def winner(x):
    if x == 0:
        return 0
    elif x > 0:
        return 1
    else:
        return 2

In [31]:
scores = []
for i in range(0, len(df['url']), 3):
    x = list(df['url'][i:i + 3])
    print(x)
    with concurrent.futures.ThreadPoolExecutor() as executor:
        r = executor.map(get_winner, x)
    for result in r:
        scores.append(result)

['https://www.fotmob.com/match/3888550/matchfacts/puebla-vs-fc-juarez', 'https://www.fotmob.com/match/3888551/matchfacts/mazatlan-fc-vs-cf-america', 'https://www.fotmob.com/match/3900373/matchfacts/fc-utrecht-vs-ajax']
['https://www.fotmob.com/match/3904422/matchfacts/nantes-vs-toulouse', 'https://www.fotmob.com/match/3902828/matchfacts/dundee-united-vs-celtic', 'https://www.fotmob.com/match/3916965/matchfacts/gent-vs-royal-antwerp']
['https://www.fotmob.com/match/3887414/matchfacts/randers-fc-vs-aalborg-bk', 'https://www.fotmob.com/match/3887416/matchfacts/viborg-vs-lyngby', 'https://www.fotmob.com/match/3905446/matchfacts/lugano-vs-st.-gallen']
['https://www.fotmob.com/match/3900374/matchfacts/excelsior-vs-psv-eindhoven', 'https://www.fotmob.com/match/3900375/matchfacts/fc-volendam-vs-fc-twente', 'https://www.fotmob.com/match/3900963/matchfacts/aston-villa-vs-west-ham-united']
['https://www.fotmob.com/match/3900971/matchfacts/wolverhampton-wanderers-vs-newcastle-united', 'https://www

['https://www.fotmob.com/match/3888623/matchfacts/necaxa-vs-leon', 'https://www.fotmob.com/match/3900986/matchfacts/everton-vs-liverpool', 'https://www.fotmob.com/match/3902849/matchfacts/celtic-vs-rangers']
['https://www.fotmob.com/match/3917967/matchfacts/mallorca-vs-girona', 'https://www.fotmob.com/match/3919120/matchfacts/fiorentina-vs-juventus', 'https://www.fotmob.com/match/3903581/matchfacts/bayer-leverkusen-vs-sc-freiburg']
['https://www.fotmob.com/match/3903586/matchfacts/vfl-bochum-vs-werder-bremen', 'https://www.fotmob.com/match/3903582/matchfacts/union-berlin-vs-bayern-m%C3%BCnchen', 'https://www.fotmob.com/match/3903585/matchfacts/wolfsburg-vs-1.-fc-k%C3%B6ln']
['https://www.fotmob.com/match/3900983/matchfacts/brentford-vs-leeds-united', 'https://www.fotmob.com/match/3900985/matchfacts/chelsea-vs-west-ham-united', 'https://www.fotmob.com/match/3900988/matchfacts/newcastle-united-vs-crystal-palace']
['https://www.fotmob.com/match/3900990/matchfacts/tottenham-hotspur-vs-fulh